In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import statistics as stats
import csv
from kuramoto import Kuramoto, plot_phase_coherence, plot_activity

In [ ]:
### Implement a method in the Kuramoto class to generate databases of colors